In [0]:
# Importing Libraries

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

In [0]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [0]:
# Data directory
DATADIR = '/content/drive/My Drive/ProjectHAR/HAR/UCI_HAR_Dataset'

In [0]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [0]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/My Drive/ProjectHAR/HAR/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    #/content/drive/My Drive/HumanActivityRecognition.zip (Unzipped Files)/HAR/UCI_HAR_Dataset/train/subject_train.txt
    #/content/drive/My Drive/HumanActivityRecognition.zip (Unzipped Files) (1)/HAR/UCI_HAR_Dataset/test/Inertial Signals/body_acc_x_test.txt
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [0]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/content/drive/My Drive/ProjectHAR/HAR/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [0]:
def load_data():
    
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [0]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [0]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [0]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [0]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [0]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [0]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [0]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [0]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [0]:
X_train.shape

(7352, 128, 9)

In [0]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

## **LSTM Architecture**

## **1-Layer LSTM with 32 units**

In [0]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/30




7352/7352 [==============================] - 33s 5ms/step - loss: 1.3108 - acc: 0.4396 - val_loss: 1.1289 - val_acc: 0.4774
Epoch 2/30
7352/7352 [==============================] - 32s 4ms/step - loss: 0.9592 - acc: 0.5887 - val_loss: 0.8989 - val_acc: 0.5860
Epoch 3/30
7352/7352 [==============================] - 32s 4ms/step - loss: 0.8008 - acc: 0.6419 - val_loss: 0.8027 - val_acc: 0.5911
Epoch 4/30
7352/7352 [==============================] - 32s 4ms/step - loss: 0.6974 - acc: 0.6533 - val_loss: 0.8914 - val_acc: 0.5816
Epoch 5/30
7352/7352 [==============================] - 32s 4ms/step - loss: 0.6501 - acc: 0.6740 - val_loss: 0.7635 - val_acc: 0.6077
Epoch 6/30
7352/7352 [==============================] - 31s 4ms/step - loss: 0.5935 - acc: 0.6910 - val_loss: 0.7337 - val_acc: 0.6661
Epoch 7/30
7352/7352 [====================

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 512        0        25        0                   0   
SITTING                  3      410        75        0                   0   
STANDING                 0       87       445        0                   0   
WALKING                  0        0         0      481                   2   
WALKING_DOWNSTAIRS       0        0         0        0                 382   
WALKING_UPSTAIRS         0        0         0        2                  18   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            3  
STANDING                           0  
WALKING                           13  
WALKING_DOWNSTAIRS                38  
WALKING_UPSTAIRS                 451  


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 4s 2ms/step


In [0]:
score

[0.3087582236972612, 0.9097387173396675]

Here the test accuracy is 91%

## **1-Layer LSTM with 128 units**

In [0]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
# Initiliazing the sequential model
model = Sequential()
model.add(LSTM(128,return_sequences=True,input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.7))
# Configuring the parameters
model.add(LSTM(128, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.8))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128, 128)          70656     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 774       
Total params: 203,014
Trainable params: 203,014
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=20)

Train on 7352 samples, validate on 2947 samples
Epoch 1/20
7352/7352 [==============================] - 142s 19ms/step - loss: 1.4266 - acc: 0.3531 - val_loss: 1.4238 - val_acc: 0.3482
Epoch 2/20
7352/7352 [==============================] - 138s 19ms/step - loss: 1.3791 - acc: 0.3626 - val_loss: 1.5759 - val_acc: 0.3308
Epoch 3/20
7352/7352 [==============================] - 138s 19ms/step - loss: 1.4487 - acc: 0.3532 - val_loss: 1.4030 - val_acc: 0.3461
Epoch 4/20
7352/7352 [==============================] - 139s 19ms/step - loss: 1.3371 - acc: 0.3789 - val_loss: 1.3970 - val_acc: 0.3448
Epoch 5/20
7352/7352 [==============================] - 139s 19ms/step - loss: 1.2915 - acc: 0.3974 - val_loss: 1.2594 - val_acc: 0.4330
Epoch 6/20
7352/7352 [==============================] - 139s 19ms/step - loss: 1.2958 - acc: 0.3970 - val_loss: 1.3881 - val_acc: 0.3590
Epoch 7/20
7352/7352 [==============================] - 139s 19ms/step - loss: 1.0841 - acc: 0.4999 - val_loss: 0.9535 - val_acc: 

## **HyperParameter tuning**

In [0]:
#Hyperparameter tuning of number of Lstm units
from keras.optimizers import Adam,RMSprop,SGD
def best_hyperparameters(hidden):

    model = Sequential()
    # Configuring the parameters
    model.add(LSTM(hidden, input_shape=(timesteps, input_dim)))
    # Adding a dropout layer
    model.add(Dropout(0.5))
    # Adding a dense output layer with sigmoid activation
    model.add(Dense(n_classes, activation='sigmoid'))


    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='rmsprop')
    
    return model

In [0]:

hidden = [16,32,48,64]

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=best_hyperparameters, epochs=epochs, batch_size=batch_size, verbose=0)
param_grid = dict(hidden=hidden)

# if you are using CPU
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
# if you are using GPU dont use the n_jobs parameter

grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








In [0]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.907916 using {'hidden': 48}
0.811752 (0.040857) with: {'hidden': 16}
0.904652 (0.022746) with: {'hidden': 32}
0.907916 (0.013044) with: {'hidden': 48}
0.875544 (0.037676) with: {'hidden': 64}


After applying hyperparameter tuning on the number of hidden layers, hidden layers of 48 has the better accuracy of 90.7%

## **Hyperparameter tuning of the dropout rates**

In [0]:
def best_hyperparameters(dp):

    model = Sequential()
    # Configuring the parameters
    model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
    # Adding a dropout layer
    model.add(Dropout(dp))
    # Adding a dense output layer with sigmoid activation
    model.add(Dense(n_classes, activation='sigmoid'))


    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='rmsprop')
    
    return model

In [0]:

dp = [0.2,0.3,0.4]

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=best_hyperparameters, epochs=epochs, batch_size=batch_size, verbose=0)
param_grid = dict(dp=dp)

# if you are using CPU
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
# if you are using GPU dont use the n_jobs parameter

grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [0]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.903292 using {'dp': 0.3}
0.867111 (0.059060) with: {'dp': 0.2}
0.903292 (0.035037) with: {'dp': 0.3}
0.894314 (0.015809) with: {'dp': 0.4}


The best drop out rate after hyperparametertuning is 0.3 with a accuracy of 90%

## **2- Layer LSTM with 32 units**

In [0]:
import warnings
warnings.filterwarnings("ignore")

from keras.layers.normalization import BatchNormalization
epochs = 30
batch_size = 16
n_hidden = 32 


model = Sequential() # Initiliazing the sequential model5
model.add(LSTM(n_hidden, return_sequences=True, input_shape=(timesteps, input_dim))) # First LSTM Layer
model.add(Dropout(0.5)) # Adding a dropout layer
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim))) # Second LSTM Layer
model.add(BatchNormalization()) # Adding batch normalization
model.add(Dropout(0.5)) # Adding a dropout layer
model.add(Dense(n_classes, activation='sigmoid')) # Adding a dense output layer with sigmoid activation
model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128, 32)           5376      
_________________________________________________________________
dropout_6 (Dropout)          (None, 128, 32)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 32)                8320      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32)                128       
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 198       
Total params: 14,022
Trainable params: 13,958
Non-trainable params: 64
_________________________________________________

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']) # Compiling the model
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=20)

Train on 7352 samples, validate on 2947 samples
Epoch 1/20
7352/7352 [==============================] - 72s 10ms/step - loss: 1.0729 - acc: 0.5250 - val_loss: 0.8262 - val_acc: 0.5836
Epoch 2/20
7352/7352 [==============================] - 69s 9ms/step - loss: 0.7666 - acc: 0.5944 - val_loss: 0.7741 - val_acc: 0.6186
Epoch 3/20
7352/7352 [==============================] - 69s 9ms/step - loss: 0.7430 - acc: 0.5835 - val_loss: 0.7442 - val_acc: 0.6040
Epoch 4/20
7352/7352 [==============================] - 69s 9ms/step - loss: 0.7037 - acc: 0.6213 - val_loss: 0.6824 - val_acc: 0.6261
Epoch 5/20
7352/7352 [==============================] - 69s 9ms/step - loss: 0.6536 - acc: 0.6530 - val_loss: 0.7131 - val_acc: 0.6118
Epoch 6/20
7352/7352 [==============================] - 69s 9ms/step - loss: 0.5804 - acc: 0.6692 - val_loss: 1.0870 - val_acc: 0.5847
Epoch 7/20
7352/7352 [==============================] - 69s 9ms/step - loss: 0.5361 - acc: 0.6730 - val_loss: 0.6608 - val_acc: 0.6254
Epoch 

In [0]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: nan
Test accuracy: 0.168306752629793


## **2 Layer LSTM with 64 units**

In [0]:
import warnings
warnings.filterwarnings("ignore")
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

from keras.layers.normalization import BatchNormalization
epochs = 30
batch_size = 16
n_hidden = 64 

model6 = Sequential() # Initiliazing the sequential model6
model6.add(LSTM(n_hidden, return_sequences=True, input_shape=(timesteps, input_dim))) # First LSTM Layer
model6.add(Dropout(0.5)) # Adding a dropout layer
model6.add(LSTM(n_hidden, input_shape=(timesteps, input_dim))) # Second LSTM Layer
model6.add(BatchNormalization()) # Adding batch normalization
model6.add(Dropout(0.5)) # Adding a dropout layer
model6.add(Dense(n_classes, activation='sigmoid')) # Adding a dense output layer with sigmoid activation
model6.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128, 64)           18944     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 64)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 390       
To

In [0]:
model6.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']) # Compiling the model
# Training the model
model6.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/30




7352/7352 [==============================] - 77s 11ms/step - loss: 0.9581 - acc: 0.5652 - val_loss: 0.7742 - val_acc: 0.5558
Epoch 2/30
7352/7352 [==============================] - 75s 10ms/step - loss: 0.7610 - acc: 0.5964 - val_loss: 0.8119 - val_acc: 0.5864
Epoch 3/30
7352/7352 [==============================] - 76s 10ms/step - loss: 0.7460 - acc: 0.5895 - val_loss: 0.7187 - val_acc: 0.6237
Epoch 4/30
7352/7352 [==============================] - 76s 10ms/step - loss: 0.6410 - acc: 0.5926 - val_loss: 0.9511 - val_acc: 0.6132
Epoch 5/30
7352/7352 [==============================] - 76s 10ms/step - loss: 0.5845 - acc: 0.6390 - val_loss: 0.6596 - val_acc: 0.6193
Epoch 6/30
7352/7352 [==============================] - 76s 10ms/step - loss: 0.5287 - acc: 0.6609 - val_loss: 0.7717 - val_acc: 0.6189
Epoch 7/30
7352/7352 [============

In [0]:
score = model6.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.5397744755697206
Test accuracy: 0.9110960298608755


## **2-Layer LSTM with 128 units**

In [0]:
epochs = 30
batch_size = 16
n_hidden = 128 #64 LSTM Units

model7 = Sequential() # Initiliazing the sequential model7
model7.add(LSTM(n_hidden, return_sequences=True, input_shape=(timesteps, input_dim))) # First LSTM Layer
model7.add(Dropout(0.3)) # Adding a dropout layer
model7.add(LSTM(n_hidden, input_shape=(timesteps, input_dim))) # Second LSTM Layer
model7.add(BatchNormalization()) # Adding batch normalization
model7.add(Dropout(0.3)) # Adding a dropout layer
model7.add(Dense(n_classes, activation='sigmoid')) # Adding a dense output layer with sigmoid activation
model7.summary()
model7.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']) # Compiling the model
model7.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs) # Training the model

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128, 128)          70656     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 774       
Total params: 203,526
Trainable params: 203,270
Non-trainable params: 256
______________________________________________

In [0]:
score = model7.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.3855398340146181
Test accuracy: 0.9005768578215134


Lets increase the dropout rate to 0.5

## **2-Layer LSTM with dropout rate=0.5**

In [0]:
epochs = 30
batch_size = 16
n_hidden = 128 #64 LSTM Units

model3 = Sequential() # Initiliazing the sequential model7
model3.add(LSTM(n_hidden, return_sequences=True, input_shape=(timesteps, input_dim))) # First LSTM Layer
model3.add(Dropout(0.5)) # Adding a dropout layer
model3.add(LSTM(n_hidden, input_shape=(timesteps, input_dim))) # Second LSTM Layer
model3.add(BatchNormalization()) # Adding batch normalization
model3.add(Dropout(0.5)) # Adding a dropout layer
model3.add(Dense(n_classes, activation='sigmoid')) # Adding a dense output layer with sigmoid activation
model3.summary()
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # Compiling the model
model3.fit(X_train, Y_train, batch_size=batch_size, validation_data=(X_test, Y_test), epochs=epochs) # Training the model

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 128, 128)          70656     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128, 128)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               131584    
_________________________________________________________________
batch_normalization_3 (Batch (None, 128)               512       
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 774       
Total params: 203,526
Trainable params: 203,270
Non-trainable params: 256
______________________________________________

In [0]:
score = model3.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.3208585181328503
Test accuracy: 0.9144893111638955


# **CNN Architecture**

## **ConvNet  with 32 filters**

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D,Conv1D, MaxPooling2D,MaxPooling1D
from keras import backend as K

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu',kernel_initializer='he_uniform',input_shape=(128,9)))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu',kernel_initializer='he_uniform'))
model.add(Dropout(0.6))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 126, 32)           896       
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 124, 32)           3104      
_________________________________________________________________
dropout_2 (Dropout)          (None, 124, 32)           0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 62, 32)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1984)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                99250     
_________________________________________________________________
dense_4 (Dense)              (None, 6)                

In [0]:
import keras
import keras.utils
from keras import utils as np_utils
import math
from keras import optimizers
adam = keras.optimizers.Adam(lr=0.001)

In [0]:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, Y_train,
          batch_size=16,
          epochs=20,
          verbose=1,
          validation_data=(X_test, Y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/20




7352/7352 [==============================] - 6s 755us/step - loss: 0.4242 - acc: 0.8342 - val_loss: 0.4439 - val_acc: 0.8269
Epoch 2/20
7352/7352 [==============================] - 4s 604us/step - loss: 0.1653 - acc: 0.9329 - val_loss: 0.4324 - val_acc: 0.8785
Epoch 3/20
7352/7352 [==============================] - 4s 598us/step - loss: 0.1302 - acc: 0.9470 - val_loss: 0.4259 - val_acc: 0.8850
Epoch 4/20
7352/7352 [==============================] - 4s 601us/step - loss: 0.1190 - acc: 0.9517 - val_loss: 0.4608 - val_acc: 0.8680
Epoch 5/20
7352/7352 [==============================] - 4s 603us/step - loss: 0.1162 - acc: 0.9520 - val_loss: 0.4690 - val_acc: 0.8860
Epoch 6/20
7352/7352 [==============================] - 4s 603us/step - loss: 0.1024 - acc: 0.9574 - val_loss: 0.4608 - val_acc: 0.8979
Epoch 7/20
7352/7352 [==============

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D,Conv1D, MaxPooling2D,MaxPooling1D
from keras import backend as K

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu',input_shape=(128,9)))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(Dropout(0.6))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
#model.add(Dropout(0.6))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 126, 32)           896       
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 124, 32)           3104      
_________________________________________________________________
dropout_2 (Dropout)          (None, 124, 32)           0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 122, 32)           3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 61, 32)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1952)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)               

In [0]:
from keras import optimizers
#sgd = optimizers.SGD(lr=0.01, decay=0.00005, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics = ["accuracy"])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, Y_train,
          batch_size=16,
          epochs=30,
          verbose=1,
          validation_data=(X_test, Y_test))

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 6s 843us/step - loss: 0.5358 - acc: 0.7875 - val_loss: 0.5435 - val_acc: 0.7978
Epoch 2/30
7352/7352 [==============================] - 6s 776us/step - loss: 0.1930 - acc: 0.9302 - val_loss: 0.4397 - val_acc: 0.8704
Epoch 3/30
7352/7352 [==============================] - 6s 764us/step - loss: 0.1347 - acc: 0.9442 - val_loss: 0.5029 - val_acc: 0.8935
Epoch 4/30
7352/7352 [==============================] - 6s 767us/step - loss: 0.1293 - acc: 0.9461 - val_loss: 0.4058 - val_acc: 0.8979
Epoch 5/30
7352/7352 [==============================] - 6s 750us/step - loss: 0.1254 - acc: 0.9505 - val_loss: 0.3995 - val_acc: 0.9023
Epoch 6/30
7352/7352 [==============================] - 6s 758us/step - loss: 0.1177 - acc: 0.9498 - val_loss: 0.4895 - val_acc: 0.8999
Epoch 7/30
7352/7352 [==============================] - 6s 753us/step - loss: 0.1107 - acc: 0.9494 - val_loss: 0.5276 - val_acc: 0.8867


In [0]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7236240969682106
Test accuracy: 0.9026128266033254


The model is trained and it gives a accuracy of 90%.<br>
Now I reduce the number of CONV layers and then the train the model

In [0]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu',input_shape=(128,9)))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(Dropout(0.6))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 126, 32)           896       
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 124, 32)           3104      
_________________________________________________________________
dropout_4 (Dropout)          (None, 124, 32)           0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 62, 32)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 1984)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 50)                99250     
_________________________________________________________________
dense_8 (Dense)              (None, 6)               

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.fit(X_train,Y_train, epochs=30, batch_size=16,validation_data=(X_test, Y_test), verbose=1)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 5s 724us/step - loss: 0.4910 - acc: 0.7986 - val_loss: 0.4949 - val_acc: 0.7991
Epoch 2/30
7352/7352 [==============================] - 4s 588us/step - loss: 0.1991 - acc: 0.9240 - val_loss: 0.3604 - val_acc: 0.8802
Epoch 3/30
7352/7352 [==============================] - 4s 576us/step - loss: 0.1443 - acc: 0.9438 - val_loss: 0.3079 - val_acc: 0.8921
Epoch 4/30
7352/7352 [==============================] - 4s 587us/step - loss: 0.1221 - acc: 0.9498 - val_loss: 0.3357 - val_acc: 0.8914
Epoch 5/30
7352/7352 [==============================] - 4s 584us/step - loss: 0.1185 - acc: 0.9523 - val_loss: 0.3230 - val_acc: 0.8839
Epoch 6/30
7352/7352 [==============================] - 4s 582us/step - loss: 0.1064 - acc: 0.9547 - val_loss: 0.5156 - val_acc: 0.8697
Epoch 7/30
7352/7352 [==============================] - 4s 584us/step - loss: 0.1115 - acc: 0.9528 - val_loss: 0.3729 - val_acc: 0.8901


In [0]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.3916387325668766
Test accuracy: 0.9107567017305734


We observe a improvemnet in the accuracy 91%

# **CNN Architecture 2**

Here is the paper I refred to (https://www.sciencedirect.com/science/article/pii/S0957417416302056). I made used of a similar architecture of a multi layer ConvNet

In [0]:
#Deep Convolutional and LSTM Recurrent Neural Networks for Multimodal Wearable Activity Recognition, 2016.
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D,Conv1D, MaxPooling2D,MaxPooling1D
from keras import backend as K

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=5, activation='relu',input_shape=(128,9)))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))

model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_14 (Conv1D)           (None, 124, 64)           2944      
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 120, 64)           20544     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 60, 64)            0         
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 56, 64)            20544     
_________________________________________________________________
dropout_6 (Dropout)          (None, 56, 64)            0         
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 28, 64)            0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 1792)             

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.fit(X_train,Y_train, epochs=30, batch_size=16,validation_data=(X_test, Y_test), verbose=1)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 11s 2ms/step - loss: 0.3970 - acc: 0.8434 - val_loss: 0.4124 - val_acc: 0.8761
Epoch 2/30
7352/7352 [==============================] - 10s 1ms/step - loss: 0.1575 - acc: 0.9370 - val_loss: 0.3159 - val_acc: 0.9104
Epoch 3/30
7352/7352 [==============================] - 10s 1ms/step - loss: 0.1292 - acc: 0.9487 - val_loss: 0.3791 - val_acc: 0.8853
Epoch 4/30
7352/7352 [==============================] - 10s 1ms/step - loss: 0.1298 - acc: 0.9512 - val_loss: 0.4278 - val_acc: 0.9006
Epoch 5/30
7352/7352 [==============================] - 10s 1ms/step - loss: 0.1215 - acc: 0.9508 - val_loss: 0.4364 - val_acc: 0.8839
Epoch 6/30
7352/7352 [==============================] - 10s 1ms/step - loss: 0.1065 - acc: 0.9543 - val_loss: 0.1832 - val_acc: 0.9274
Epoch 7/30
7352/7352 [==============================] - 10s 1ms/step - loss: 0.0979 - acc: 0.9581 - val_loss: 0.2354 - val_acc: 0.9182
Epoch 8

In [0]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.46918977352065117
Test accuracy: 0.9270444519850696


The model gives a good accuracy of 92.7% and test loss of 0.46 <br>
It is observed that a CNN based model performs better than LSTM.

# **ConvNet with Batch Normalization**

In [0]:
#Deep Convolutional and LSTM Recurrent Neural Networks for Multimodal Wearable Activity Recognition, 2016.
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D,Conv1D, MaxPooling2D,MaxPooling1D
from keras import backend as K

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=5, activation='relu',input_shape=(128,9)))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(BatchNormalization())

model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 124, 64)           2944      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 120, 64)           20544     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 60, 64)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 56, 64)            20544     
_________________________________________________________________
batch_normalization_4 (Batch (None, 56, 64)            256       
_________________________________________________________________
dropout_7 (Dropout)          (None, 56, 64)            0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 28, 64)          

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
model.fit(X_train,Y_train, epochs=30, batch_size=batch_size,validation_data=(X_test, Y_test), verbose=1)


Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 11s 2ms/step - loss: 0.3970 - acc: 0.8434 - val_loss: 0.4124 - val_acc: 0.8761
Epoch 2/30
7352/7352 [==============================] - 10s 1ms/step - loss: 0.1575 - acc: 0.9370 - val_loss: 0.3159 - val_acc: 0.9104
Epoch 3/30
7352/7352 [==============================] - 10s 1ms/step - loss: 0.1292 - acc: 0.9487 - val_loss: 0.3791 - val_acc: 0.8853
Epoch 4/30
7352/7352 [==============================] - 10s 1ms/step - loss: 0.1298 - acc: 0.9512 - val_loss: 0.4278 - val_acc: 0.9006
Epoch 5/30
7352/7352 [==============================] - 10s 1ms/step - loss: 0.1215 - acc: 0.9508 - val_loss: 0.4364 - val_acc: 0.8839
Epoch 6/30
7352/7352 [==============================] - 10s 1ms/step - loss: 0.1065 - acc: 0.9543 - val_loss: 0.1832 - val_acc: 0.9274
Epoch 7/30
7352/7352 [==============================] - 10s 1ms/step - loss: 0.0979 - acc: 0.9581 - val_loss: 0.2354 - val_acc: 0.9182
Epoch 8

In [0]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:',score[0])
print('Test accuracy:',score[1])

Test loss: 0.4692240974282106
Test accuracy: 0.9292128266033254


Here we get a accuracy of 93%

## **Conclusion**

In [0]:
from prettytable import PrettyTable    
x = PrettyTable()
x.field_names = ["Model","Accuracy"]
x.add_row(['1 layer LSTM with 32 units',"90.1%"])
x.add_row(['1 layer LSTM with 128 units',"60%"])
x.add_row(['1 layer LSTM with 48 units',"90.2%"])
x.add_row(['1 layer LSTM with 32 units with 0.3 dropout',"90.6%"])
x.add_row(['2 layer LSTM with 32 units',"90%"])
x.add_row(['2 layer LSTM with 64 units',"91.1%"])
x.add_row(['2 layer LSTM with 128 units',"90.8%"])
x.add_row(['2 layer LSTM with 128 units with droput rate 0.5',"91.4%"])
x.add_row(['ConvNet(2 conv layer with 32 filters)',"88%"])
x.add_row(['ConvNet(3 conv layer with 32 filter)',"90.8%"])
x.add_row(['Convnet with 64 filter',"92.6%"])
x.add_row(['Convnet with 64 filter with Batch Normalization',"93%"])
print(x)

+--------------------------------------------------+----------+
|                      Model                       | Accuracy |
+--------------------------------------------------+----------+
|            1 layer LSTM with 32 units            |  90.1%   |
|           1 layer LSTM with 128 units            |   60%    |
|            1 layer LSTM with 48 units            |  90.2%   |
|   1 layer LSTM with 32 units with 0.3 dropout    |  90.6%   |
|            2 layer LSTM with 32 units            |   90%    |
|            2 layer LSTM with 64 units            |  91.1%   |
|           2 layer LSTM with 128 units            |  90.8%   |
| 2 layer LSTM with 128 units with droput rate 0.5 |  91.4%   |
|      ConvNet(2 conv layer with 32 filters)       |   88%    |
|       ConvNet(3 conv layer with 32 filter)       |  90.8%   |
|              Convnet with 64 filter              |  92.6%   |
| Convnet with 64 filter with Batch Normalization  |   93%    |
+---------------------------------------

It is observed that CNN works better than LSTM.<br>
I experimented with variuos LSTM model but coudn't improve the accuracy<br>
At the end ConvNet with 92.9% ~ 93% accuracy works the best which is close to 94%.<br>